In [1]:
using DataFrames              # package for working with data sets
using JWAS                    # package for Bayesian regression analyses, including BayesB and BayesCπ        
using JWAS:misc               # utility functions
using Distributions       
using Plots                   # package for plotting 
using CSV
using LinearAlgebra,Statistics,Random,DelimitedFiles, DataFrames

In [2]:
phenotypes = CSV.read("Data_sp_reprod_per_parity_only_sp.csv",delim = ',',header=true, missingstring=".")
size(phenotypes)
first(phenotypes, 3)
val1 =  phenotypes[phenotypes[!,:Collection_Group] .== 1 ,:]
train1 =  phenotypes[phenotypes[!,:Collection_Group] .!== 1 ,:]

val2 =  phenotypes[phenotypes[!,:Collection_Group] .== 2 ,:]
train2 =  phenotypes[phenotypes[!,:Collection_Group] .!== 2 ,:]

val3 =  phenotypes[phenotypes[!,:Collection_Group] .== 3 ,:]
train3 =  phenotypes[phenotypes[!,:Collection_Group] .!== 3 ,:];

In [3]:
marker1 = CSV.read("GBLUP_train1_marker_effects_SP_ration.txt", header = true, delim = ",")
marker2 = CSV.read("GBLUP_train2_marker_effects_SP_ration.txt", header = true, delim = ",")
marker3 = CSV.read("GBLUP_train3_marker_effects_SP_ration.txt", header = true, delim = ",")
ids = CSV.read("markerID.csv", header = true, delim = ",", missingstring=".");

In [4]:
genotypes = CSV.read("Genotype_AnimalID_imputed_final_ASReml_order_numerical.mkr", header=true, delim=' ');

In [5]:
alleleFreq = mean(Matrix(genotypes[:,2:45537]), dims=1)
genotypes2 = genotypes[:,2:45537] .- alleleFreq;
first(genotypes2, 5)

,1_242598,WU_10.2_1_266158,ASGA0000005,ASGA0000014,ASGA0000021,WU_10.2_1_722623
,Float64,Float64,Float64,Float64,Float64,Float64
1,-1.00338,-0.589628,0.234498,-1.00338,0.768884,-0.310034
2,-1.00338,-0.589628,0.234498,-1.00338,0.768884,-0.310034
3,-0.00338219,-0.589628,0.234498,-0.00338219,-0.231116,0.689966
4,-1.00338,-0.589628,0.234498,-1.00338,0.768884,-0.310034
5,-0.00338219,0.410372,0.234498,-0.00338219,-0.231116,-0.310034


# Prediction for MHC

In [10]:
ids_MHC=ids[1:84,1];  ### 84 markers in MHC region (from 23 Mb to 26 Mb)

In [40]:
marker1_MHC= marker1[:,[x in ids_MHC for x in String.(names(marker1))]]
marker1_MHC = transpose(Matrix(mean(Matrix(marker1_MHC), dims=1)))
marker2_MHC= marker2[:,[x in ids_MHC for x in String.(names(marker2))]]
marker2_MHC = transpose(Matrix(mean(Matrix(marker2_MHC), dims=1)))
marker3_MHC= marker3[:,[x in ids_MHC for x in String.(names(marker3))]]
marker3_MHC = transpose(Matrix(mean(Matrix(marker3_MHC), dims=1)))
genotypes_MHC = genotypes2[:, [x in ids_MHC for x in String.(names(genotypes2))]]

InterruptException: InterruptException:

In [30]:
first(genotypes_MHC, 5)

,1_242598,WU_10.2_1_266158,ASGA0000005,ASGA0000014,ASGA0000021,WU_10.2_1_722623
,Float64,Float64,Float64,Float64,Float64,Float64
1,-1.00338,-0.589628,0.234498,-1.00338,0.768884,-0.310034
2,-1.00338,-0.589628,0.234498,-1.00338,0.768884,-0.310034
3,-0.00338219,-0.589628,0.234498,-0.00338219,-0.231116,0.689966
4,-1.00338,-0.589628,0.234498,-1.00338,0.768884,-0.310034
5,-0.00338219,0.410372,0.234498,-0.00338219,-0.231116,-0.310034


In [69]:
#ebv1 = Matrix(genotypes) * Matrix(mean_mkr)
#ebv1 = DataFrame(ebv1)
#insertcols!(ebv1, 1, :ID => genotypes[:,:AnimalID])
#first(ebv1,5)

,x1
,Float64
1,-0.553979
2,-0.278123
3,0.160126
4,-0.256819
5,0.247842


In [70]:
#ebv_right = CSV.read("EBV_SP_ration_train3.txt",header=true, missingstring=".")
#first(ebv_right, 5)

,ID,EBV,PEV
,Int64,Float64,Float64
1,43108826,-0.553979,0.0234895
2,43108854,-0.278123,0.0243226
3,43108856,0.160126,0.0284833
4,43108864,-0.256819,0.0234532
5,43108920,0.247842,0.0298538


In [31]:
ebv1 = Matrix(genotypes_MHC) * Matrix(marker1_MHC)
ebv1 = DataFrame(ebv1)
insertcols!(ebv1, 1, :ID => genotypes[:,:AnimalID])
ebv2 = Matrix(genotypes_MHC) * Matrix(marker2_MHC)
ebv2 = DataFrame(ebv2)
insertcols!(ebv2, 1, :ID => genotypes[:,:AnimalID])
ebv3 = Matrix(genotypes_MHC) * Matrix(marker3_MHC)
ebv3 = DataFrame(ebv3)
insertcols!(ebv3, 1, :ID => genotypes[:,:AnimalID]);

In [39]:
first(ebv1,5)

,ID,x1
,Int64,Float64
1,43108826,-0.192812
2,43108854,-0.237136
3,43108856,0.010054
4,43108864,-0.233608
5,43108920,0.387978


In [32]:
ids_val1 = val1[!,1]
ids_val2 = val2[!,1]
ids_val3 = val3[!,1];

In [33]:
ebv_val1 = ebv1[[x in ids_val1 for x in ebv1[:,:ID]], :]
ebv_val2 = ebv2[[x in ids_val2 for x in ebv2[:,:ID]], :]
ebv_val3 = ebv3[[x in ids_val3 for x in ebv3[:,:ID]], :];

In [34]:
insertcols!(ebv_val1, 3, :SP_ratio => val1[!,4] )
insertcols!(ebv_val2, 3, :SP_ratio => val2[!,4] )
insertcols!(ebv_val3, 3, :SP_ratio => val3[!,4] );

In [35]:
cor1 = cor(ebv_val1[!,3], ebv_val1[!,2])
cor2 = cor(ebv_val2[!,3], ebv_val2[!,2])
cor3 = cor(ebv_val3[!,3], ebv_val3[!,2]);
[cor1 cor2 cor3]

1×3 Array{Float64,2}:
 0.411534  0.767141  0.782144

In [17]:
cor_all = (size(val1,1)*cor1 + size(val2,1)*cor2 + size(val3,1)*cor3)/(size(val1,1) + size(val2,1) + size(val3,1));

In [18]:
acc = cor_all/sqrt(0.34)
std_acc = std([cor1,cor2,cor3])/sqrt(0.34);

In [19]:
[acc std_acc]

1×2 Array{Float64,2}:
 0.598265  0.00719923

# Prediciton for REST now 

In [15]:
ids_MHC=ids[1:45379,2];### 45,379 markers in REST region (removed from 21 Mb to 28 Mb )
ids_MHC;

In [16]:
marker1_MHC= marker1[:,[x in ids_MHC for x in String.(names(marker1))]]
marker1_MHC = transpose(Matrix(mean(Matrix(marker1_MHC), dims=1)))
marker2_MHC= marker2[:,[x in ids_MHC for x in String.(names(marker2))]]
marker2_MHC = transpose(Matrix(mean(Matrix(marker2_MHC), dims=1)))
marker3_MHC= marker3[:,[x in ids_MHC for x in String.(names(marker3))]]
marker3_MHC = transpose(Matrix(mean(Matrix(marker3_MHC), dims=1)))

45379×1 Transpose{Float64,Array{Float64,2}}:
  1.1721617577987476e-5 
  9.497572768916633e-6  
  8.123497120039534e-5  
 -1.9290438540555178e-5 
  4.0489000997307466e-5 
 -5.490330128490659e-5  
 -2.7341217131990035e-5 
  1.5548032881991503e-5 
 -1.7338433742569676e-6 
  5.784905240671039e-5  
  2.3449133151029982e-5 
 -0.00012889772115172232
 -4.387693137113376e-5  
  ⋮                     
  1.941046640730475e-5  
  5.6929500587791024e-5 
 -1.1856608687695102e-5 
 -7.198508596498331e-6  
  0.0                   
 -1.5767966575489013e-5 
 -1.9174182435400174e-5 
 -0.00023086675518678594
 -0.0008018581514695296 
 -0.00046873279185272957
 -0.0005583290800682188 
 -0.00011558182927843697

In [17]:
genotypes_MHC = genotypes2[:, [x in ids_MHC for x in String.(names(genotypes2))]];

In [18]:
ebv1 = Matrix(genotypes_MHC) * Matrix(marker1_MHC)
ebv1 = DataFrame(ebv1)
insertcols!(ebv1, 1, :ID => genotypes[:,:AnimalID])
ebv2 = Matrix(genotypes_MHC) * Matrix(marker2_MHC)
ebv2 = DataFrame(ebv2)
insertcols!(ebv2, 1, :ID => genotypes[:,:AnimalID])
ebv3 = Matrix(genotypes_MHC) * Matrix(marker3_MHC)
ebv3 = DataFrame(ebv3)
insertcols!(ebv3, 1, :ID => genotypes[:,:AnimalID]);

In [19]:
ids_val1 = val1[!,1]
ids_val2 = val2[!,1]
ids_val3 = val3[!,1];

In [20]:
ebv_val1 = ebv1[[x in ids_val1 for x in ebv1[:,:ID]], :]
ebv_val2 = ebv2[[x in ids_val2 for x in ebv2[:,:ID]], :]
ebv_val3 = ebv3[[x in ids_val3 for x in ebv3[:,:ID]], :];

In [21]:
insertcols!(ebv_val1, 3, :SP_ratio => val1[!,4] )
insertcols!(ebv_val2, 3, :SP_ratio => val2[!,4] )
insertcols!(ebv_val3, 3, :SP_ratio => val3[!,4] );

In [22]:
cor1 = cor(ebv_val1[!,3], ebv_val1[!,2])
cor2 = cor(ebv_val2[!,3], ebv_val2[!,2])
cor3 = cor(ebv_val3[!,3], ebv_val3[!,2])
[cor1 cor2 cor3]

1×3 Array{Float64,2}:
 0.234062  0.214224  0.187953

In [23]:
cor_all = (size(val1,1)*cor1 + size(val2,1)*cor2 + size(val3,1)*cor3)/(size(val1,1) + size(val2,1) + size(val3,1));
acc = cor_all/sqrt(0.34)
std_acc = std([cor1,cor2,cor3])/sqrt(0.34);
[acc std_acc]

1×2 Array{Float64,2}:
 0.36909  0.0396659